Domain Decomposition with minimal overlap
===

With domain decomposition (DD) methods one splits a large problem on a big domain $\Omega$ into smaller problems on sub-domains $\Omega_i$.
One advantage is that direct factorization on each sub-domain is cheaper than factorization of the global problem. Assume that the global problem is of size $N$, and we use $m$ sub-domains. Assume the  (sparse) factorization of a problem of size $n$ has costs $O(n^\alpha)$. Then the advantage is

$$
m \, \big( \tfrac{N}{m} \big)^\alpha = \frac{1}{m^{\alpha-1}} N^\alpha \leq N^\alpha
$$

For sparse direct solvers based on nested dissection $\alpha = 1.5$ for 2D problems, and $\alpha = 2$ for 3D.

We sub-divide the unit-square into $m_x \times m_y$ sub-domains. On the lower boundary we assign a boundary condition label:

In [ ]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.geom2d import CSG2d, Rectangle

geo = CSG2d()

mx, my = 3,3
for i in range(mx): 
    for j in range(my):
        rect = Rectangle(pmin=(i/mx,j/my), \
                         pmax=((i+1)/mx,(j+1)/mx), \
                         mat='mat'+str(i)+str(j), \
                         bc = 'default', \
                         bottom = 'bot' if j == 0 else None)
                  
        geo.Add(rect)
        
mesh = Mesh(geo.GenerateMesh(maxh=0.05))
# print (mesh.GetMaterials())
# print (mesh.GetBoundaries())
Draw (mesh)

setup a standard problem, Dirichlet b.c. on the bottom boundary:

In [ ]:
fes = H1(mesh, order=1, dirichlet='bot')
print ("ndof =", fes.ndof)
u,v = fes.TnT()
a = BilinearForm(grad(u)*grad(v)*dx).Assemble()

We build a precontioner as the sum of inverses on sub-domains:

1. We iterate over the individual sub-domains. The function `mesh.Materials` returns one volume region, which can be split into a list of regions. 
99. A finite element space can provide a `BitArray` marking the dofs of a certain region. Logical and with the non-Dirichet dofs.
99. The `Inverse(domaindofs)` takes the sub-matrix corresponding to the domain dofs, inverts it with a sparse direct solver, and inserts the small inverse into the large by padding with zeros. The costs are  $O(N + N_{\text{used}}^\alpha)$, where the constant for the $O(N)$ term is very small. 
99. The final preconditioner is the sum of sub-domain inverses.

In [ ]:
pre = None
for dom in mesh.Materials(".*").Split():
    domaindofs = fes.GetDofs(dom) & fes.FreeDofs()
    # print (domaindofs)
    # print ("num dofs:", domaindofs.NumSet())
    invi = a.mat.Inverse(domaindofs)
    pre = invi if pre == None else pre + invi
    
from ngsolve.la import EigenValues_Preconditioner
lami = list(EigenValues_Preconditioner(mat=a.mat, pre=pre))
print ("condition number = ", lami[-1]/lami[0])

To check the solution:

In [ ]:
f = LinearForm(1*v*dx).Assemble()
gfu = GridFunction(fes)
gfu.vec.data = CGSolver(mat=a.mat, pre=pre) * f.vec
Draw (gfu)

Experiment with number of subdomains, and mesh-size. You should observe a condition number $O( h^{-1} H^{-1} )$, where $H$ is the sub-domain size, and $H$ is the element-size.

Analysis of the method
---

We apply the ASM theory, where sub-spaces are

$$
V_i = \operatorname{span} \{ \varphi_i  : \text{dof}_i \text{ supported in } \overline{\Omega_i} \}
$$

By the ASM Lemma, the preconditioner norm is

$$
\| u \|_{C_{ASM}}^2 = \inf_{u = \sum u_i : u_i \in V_i } \sum \| u_i \|_A^2
$$

Since one sub-space $V_i$ has overlap with at most 9 spaces $V_j$, there immediately follows 

$$
\| u \|_A^2 \leq 9 \, \| u \|_{C_{ASM}}^2
$$

and 

$$
\lambda_{max} (C_{ASM}^{-1} A) \leq 9
$$


For the other bound we have to find a decomposition, and bound the norm:

**Lemma** Let $u \in H^1(\Omega)$, and $u_i = u$ in $\Omega_i$, and all other dofs set to zero. Then

$$
\| u_i \|_{H_1(\Omega)}^2 \preceq \tfrac{1}{hH} \| u \|_{H^1(\Omega_i)}^2
$$

*Proof:* Follows from

$$
\| u_i \|_{L_2(\partial \Omega)}^2  \preceq \frac{1}{H} \| u \|_{H^1(\Omega)}^2
$$

and 

$$
\| u_i \|_{H^1(\Omega \setminus \Omega_i)}^2 \preceq \frac{1}{h} \| u_i \|_{L_2(\partial \Omega_i)}^2 
$$

<div style="text-align: right"> $\Box$</div>



Choosing this $u_i$ for every sub-domain does not lead to a decomposition $u = \sum{u_i}$ since dofs at the interface contribute to several sub-space functions. 
So, we have to divide each coefficient by the number of sub-domains it belongs to, to obtain $\tilde u_i$. 


In [ ]:
gfi = GridFunction(fes)
domi = mesh.Materials(".*").Split()[4]
proj = Projector(fes.GetDofs(domi), True)
gfi.vec.data = proj * gfu.vec
Draw (gfi)

Adding a coarse grid space
---
We build a coarse space of functions being constant in each sub-domain

In [ ]:
mv = MultiVector(gfu.vec, 0)
proj = Projector(fes.FreeDofs(),True)
for dom in mesh.Materials(".*").Split():
    consti = GridFunction(fes)
    consti.Set(1, definedon=dom)
    proj.Project(consti.vec)
    # Draw (consti, mesh)
    mv.Append (consti.vec)
print (len(mv))

In [ ]:
a0 = InnerProduct(mv, a.mat * mv)
# print (a0)
inva0 = BaseMatrix(a0.I)
E0 = BaseMatrix(mv)
coarsegrid = E0 @ inva0 @ E0.T

In [ ]:
pre2 = pre + coarsegrid
lami = list(EigenValues_Preconditioner(mat=a.mat, pre=pre2))
print ("condition number = ", lami[-1]/lami[0])

In [ ]:
cnt = sum(mv)
idiag = DiagonalMatrix(cnt).Inverse(fes.FreeDofs())
for v in mv:
    v.data = (idiag*v).Evaluate()

In [ ]:
a0 = InnerProduct(mv, a.mat * mv)
# print (a0)
# print (a0 * Vector( (1,)*len(mv)))
inva0 = BaseMatrix(a0.I)
E0 = BaseMatrix(mv)
coarsegrid = E0 @ inva0 @ E0.T
pre3 = pre + coarsegrid
lami = list(EigenValues_Preconditioner(mat=a.mat, pre=pre3))
print ("condition number = ", lami[-1]/lami[0])

In [ ]:
gfu0 = GridFunction(fes)
gfu0.vec.data = coarsegrid * (a.mat * gfu.vec)
Draw (gfu0)